# Notebook for demoing Bert-based LMs

### Importing packages and loading tokenizer

In [24]:
import transformers
import torch
import torch.nn.functional as F
from collections import Counter
import csv

In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained('SZTAKI-HLT/hubert-base-cc', lowercase=True)

In [18]:
c = Counter()
with open('100k_corp.spl') as infile, open('tokenized_100k_corp.spl', 'w') as outfile:
    for line in infile:
        if line[0] == '#':
            continue
        tokens = tokenizer(line.strip(), add_special_tokens=False)
        for token in tokens['input_ids']:
            c[tokenizer.decode(token)] += 1
        outfile.write(tokenizer.decode(tokens['input_ids']))
        outfile.write('\n')

In [25]:
with open('freqs.csv', 'w') as outfile:
    csv_writer = csv.writer(outfile)
    for word, freq in sorted(c.items(), key = lambda x: x[1], reverse=True):
        csv_writer.writerow([word, freq])

## Single masked token prediction with BERT

This example uses masked language modelling, that is "given the entire sentence with some MASKs, what is the most likely word in MASK positions". The prediction uses the entire sentence to guess, not only left-to-right information.

In [4]:
model = transformers.BertForMaskedLM.from_pretrained('SZTAKI-HLT/hubert-base-cc', return_dict=True)

Some weights of the model checkpoint at SZTAKI-HLT/hubert-base-cc were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
masked_word = 'szervezetek'
print(masked_word in tokenizer.vocab)
orig_text = f'Már 67 országban engedélyezték, de a nagy nyugati szervezetek még ellenállnak, hiába derül ki egyre több tanulmányból, hogy a Szputynik nyugati oltásoknál is hatásosabb.'
text = orig_text.replace(masked_word, tokenizer.mask_token, 1)
masked_word_length = len('szervezetek')

True


In [318]:
tokenized_text = tokenizer(text, return_tensors='pt')
mask_index = torch.where(tokenized_text["input_ids"][0] == tokenizer.mask_token_id)
output = model(**tokenized_text)
logits = output.logits
softmax = F.softmax(logits, dim = -1)
mask_word = softmax[0, mask_index, :]

In [319]:
def topn_fixed_length(mask_word, top_n, dim, word_length, tokenizer, check_word_length=True):
    order = torch.argsort(mask_word, dim=1, descending=True)[0]
    possible_words = []
    for token_id in order:
        token = tokenizer.decode([token_id])
        if check_word_length:
            if len(token) == word_length:
                possible_words.append(token)
        else:
            possible_words.append(token)
        if len(possible_words) == top_n:
            break
    return possible_words

In [320]:
n = 10
top_n = topn_fixed_length(mask_word, n, dim=1, word_length=masked_word_length, tokenizer=tokenizer, check_word_length=False)

In [321]:
for token in top_n:
    new_sentence = text.replace(tokenizer.mask_token, f'_{token}_')
    print(new_sentence)

Már 67 országban engedélyezték, de a nagy nyugati _gyártók_ még ellenállnak, hiába derül ki egyre több tanulmányból, hogy a Szputynik nyugati oltásoknál is hatásosabb.
Már 67 országban engedélyezték, de a nagy nyugati _országok_ még ellenállnak, hiába derül ki egyre több tanulmányból, hogy a Szputynik nyugati oltásoknál is hatásosabb.
Már 67 országban engedélyezték, de a nagy nyugati _cégek_ még ellenállnak, hiába derül ki egyre több tanulmányból, hogy a Szputynik nyugati oltásoknál is hatásosabb.
Már 67 országban engedélyezték, de a nagy nyugati _államok_ még ellenállnak, hiába derül ki egyre több tanulmányból, hogy a Szputynik nyugati oltásoknál is hatásosabb.
Már 67 országban engedélyezték, de a nagy nyugati _##ak_ még ellenállnak, hiába derül ki egyre több tanulmányból, hogy a Szputynik nyugati oltásoknál is hatásosabb.
Már 67 országban engedélyezték, de a nagy nyugati _vállalatok_ még ellenállnak, hiába derül ki egyre több tanulmányból, hogy a Szputynik nyugati oltásoknál is hatás

## Multiple masked token prediction with BERT

In [220]:
tokenizer = transformers.AutoTokenizer.from_pretrained('SZTAKI-HLT/hubert-base-cc', lowercase=True)
model = transformers.BertForMaskedLM.from_pretrained('SZTAKI-HLT/hubert-base-cc', return_dict=True)

Some weights of the model checkpoint at SZTAKI-HLT/hubert-base-cc were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [268]:
words = ['országban', 'nagy', 'hiába', 'ellenállnak', 'hatásos']
orig_text = f'Már 67 országban engedélyezték, de a nagy nyugati szervezetek még ellenállnak, hiába derül ki egyre több tanulmányból, hogy a Szputynik nyugati oltásoknál is hatásosabb.'
text = orig_text
for word in words:
    text = text.replace(word, tokenizer.mask_token, 1)
masked_word_lengths = [len(word) for word in words]

In [269]:
tokenized_text = tokenizer(text, return_tensors='pt')
mask_index = torch.where(tokenized_text["input_ids"][0] == tokenizer.mask_token_id)[0]
output = model(**tokenized_text)
logits = output.logits
softmax = F.softmax(logits, dim = -1)

In [270]:
mask_words = softmax[0, mask_index, :]


In [271]:
torch.argsort(mask_words, dim=1, descending=True)

tensor([[ 6892,  3948, 25734,  ..., 21669, 18757,  4501],
        [ 2218,  3695,  3701,  ...,  8101,  5750, 13191],
        [ 2093, 26218, 27898,  ..., 22064, 19224,  9589],
        [ 7388,  3131,  2489,  ...,  6183, 31461, 28235],
        [17554, 10192,  4494,  ..., 15983, 25276, 16085]])

In [272]:
order = torch.argsort(mask_words, dim=1, descending=True)

In [273]:
def increment_list_index(lst: list, index) -> list:
    new_lst = lst.copy()
    new_lst[index] += 1
    return new_lst

In [274]:
top_n = 30
idx = order[:, 0]
top_pairs = [[0 for id_ in idx]] # in relation to argsort
dims = range(order.shape[0])
values = mask_words[dims, idx]
last_product = torch.prod(values)
while len(top_pairs) < top_n:
    candidates = [increment_list_index(top_pairs[-1], dim) for dim in dims]
    candidate_values = [torch.prod(mask_words[dims, order[dims, candidate]]) for candidate in candidates]
    best_candidate = torch.argmax(torch.Tensor(candidate_values))
    top_pairs.append(candidates[best_candidate])
    

In [288]:
def tup_generator(mask_words, dims = None):
    """
    mask_words the softmax on the logits, dims is an iterable containing the dimensions along which we search
    """
    if dims is None:
        dims = range(softmax.shape[0])
    order = torch.argsort(mask_words, dim=1, descending=True)
    idx = order[:, 0]
    top_pairs = [[0 for id_ in idx]] # in relation to argsort
    values = mask_words[dims, idx]
    last_product = torch.prod(values)
    while True:
        candidates = [increment_list_index(top_pairs[-1], dim) for dim in dims]
        candidate_values = [torch.prod(mask_words[dims, order[dims, candidate]]) for candidate in candidates]
        best_candidate = torch.argmax(torch.Tensor(candidate_values))
        top_pairs.append(candidates[best_candidate])
        yield top_pairs[-1]

In [313]:
gen = tup_generator(mask_words)
dims = range(mask_words.shape[0])
order = torch.argsort(mask_words, dim=1, descending=True)
marked_text = orig_text
for word in words:
    marked_text = marked_text.replace(word, f'<|{word}|>', 1)
print(marked_text)
for i, idx in enumerate(gen):
    token_tup = order[dims, idx]
    new_sentence = text
    for token_id in token_tup:
        token = tokenizer.decode([token_id])
        new_sentence = new_sentence.replace(tokenizer.mask_token, f'_{token}_', 1)
    print(new_sentence)
    if i > 100:
        break

Már 67 <|országban|> engedélyezték, de a <|nagy|> nyugati szervezetek még <|ellenállnak|>, <|hiába|> derül ki egyre több tanulmányból, hogy a Szputynik nyugati oltásoknál is <|hatásos|>abb.
Már 67 _országban_ engedélyezték, de a _nagy_ nyugati szervezetek még _vizsgálják_, _hiába_ derül ki egyre több tanulmányból, hogy a Szputynik nyugati oltásoknál is _hatásos_abb.
Már 67 _országban_ engedélyezték, de a _nagy_ nyugati szervezetek még _küzdenek_, _hiába_ derül ki egyre több tanulmányból, hogy a Szputynik nyugati oltásoknál is _hatásos_abb.
Már 67 _éve_ engedélyezték, de a _nagy_ nyugati szervezetek még _küzdenek_, _hiába_ derül ki egyre több tanulmányból, hogy a Szputynik nyugati oltásoknál is _hatásos_abb.
Már 67 _államban_ engedélyezték, de a _nagy_ nyugati szervezetek még _küzdenek_, _hiába_ derül ki egyre több tanulmányból, hogy a Szputynik nyugati oltásoknál is _hatásos_abb.
Már 67 _államban_ engedélyezték, de a _nagy_ nyugati szervezetek még _keresik_, _hiába_ derül ki egyre több

In [216]:
marked_text = orig_text
for word in words:
    marked_text = marked_text.replace(word, f'<|{word}|>', 1)
print(marked_text)
for token_tup in [order[dims, pair] for pair in top_pairs]:
    new_sentence = text
    for token_id in token_tup:
        token = tokenizer.decode([token_id])
        new_sentence = new_sentence.replace(tokenizer.mask_token, f'_{token}_', 1)
    print(new_sentence)

Már 67 <|országb<|a|>n|> engedélyezték, de a <|nagy|> nyugati szervezetek még <|ellenállnak|>, <|hiába|> derül ki egyre tö<|bb|> tanulmányból, hogy a Szputynik nyugati oltásoknál is <|hatásos|>abb.
Már 67 _országban_ engedélyezték, de _a_ _a_ nyugati szervezetek még _vizsgálják_, _hiába_ derül ki egyre tö_több_ tanulmányból, hogy a Szputynik nyugati oltásoknál is _hatásos_abb.
Már 67 _országban_ engedélyezték, de _a_ _a_ nyugati szervezetek még _vizsgálják_, _hiába_ derül ki egyre tö_külföldi_ tanulmányból, hogy a Szputynik nyugati oltásoknál is _hatásos_abb.
Már 67 _országban_ engedélyezték, de _a_ _a_ nyugati szervezetek még _nem_, _hiába_ derül ki egyre tö_külföldi_ tanulmányból, hogy a Szputynik nyugati oltásoknál is _hatásos_abb.
Már 67 _országban_ engedélyezték, de _a_ _a_ nyugati szervezetek még _küzdenek_, _hiába_ derül ki egyre tö_külföldi_ tanulmányból, hogy a Szputynik nyugati oltásoknál is _hatásos_abb.
Már 67 _éve_ engedélyezték, de _a_ _a_ nyugati szervezetek még _küzdene

In [90]:
numbers = [] * order.shape[0]

In [147]:
[1, 2, 3][torch.argmax(torch.Tensor(candidate_values))]

2

In [120]:
mask_words.shape

torch.Size([2, 32001])

In [99]:
softmax[order[:, 0]]

IndexError: index 4022 is out of bounds for dimension 0 with size 1

In [123]:
mask_words[range(order.shape[0]), order[:, 0]]

tensor([0.7228, 0.4612], grad_fn=<IndexBackward>)

In [108]:
mask_words[(0, 0), (0, 1)]

tensor([2.8730e-12, 1.5070e-06], grad_fn=<IndexBackward>)

In [112]:
tuple(range(order.shape[0]))

(0, 1)

In [118]:
order[:, 0].tolist()

[6892, 4022]